# Statistics in Lie groups

> **_Tip:_** Launch live version of this tutorial: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/morphomatics/morphomatics.github.io/HEAD?filepath=docs%2Ftutorials%2Ftutorial_biinvariant_statistics.ipynb)

## Bi-invariant similarity measures
Given two distributions of samples, it is often necessary to quantify the difference between them.
Two popular indices that can be used for data from Euclidean spaces are the Hotelling $T^2$ statistic and Bhattacharyya
distance. While the first quantifies differences between the means only, the latter takes also differing covariance
structures into account.
Remarkably, both can be generalized to data from Lie groups in a way that respects
the group's fundamental properties: The _bi-invariant_ Hotelling $T^2$ statistic and Bhattacharyya distance are
invariant under translations of the data from both left and right. For shape analysis
this leads (amongst others) to an analysis that is independent of
the choice of reference.
For the definitions see **[Bi-invariant Two-Sample Tests in Lie Groups for Shape Analysis.](https://arxiv.org/abs/2008.12195)**
In this paper, there is also shown how a bi-invariant two sample test for differences in mean shape can
be constructed from these notions.

## Example
In the following, we show how to use bi-invariant similarity measures in _Morphomatics_. For this, we choose the Lie group
$\text{SE}(3)$ of rigid motions in 3-dimensional space. For bi-invariant statistics in $\text{SE}(3)$
we need a non-metric (i.e., non-Riemannian) structure. When using it, a geodesic $\gamma$ passing through a matrix $A \in \text{SE}(3)$
with tangent vector $X \in \mathfrak{se}(3)$ is given by the matrix exponential:

$$
\gamma(t) = Ae^{tX}.
$$

Since the matrix exponential is a fundamental _group_ property of $\text{SE}(3)$, this gives the desired
connection of group and geometric properties. (This works—far more generally—in any finite-dimensional Lie group.)

Thus, we can create two sample sets $E, F \subset \text{SE}(3)$ with the identity matrix $I \in \text{SE}(3)$ as mean as follows
('''G.group.exp''' is the matrix exponential).

In [7]:
%env JAX_PLATFORM_NAME=cpu

import jax
import jax.numpy as jnp

from morphomatics.manifold import SE3
from morphomatics.stats import BiinvariantStatistics

# initialize Lie group of rigid motions
G = SE3()
# initialize module for bi-invariant statistics
bistat = BiinvariantStatistics(G)
# identity matrix
I = G.group.identity

# sample 2 data sets around I
E = []
F = []
for _ in range(8):
    # random tangent vector
    v = G.randvec(I)
    # shoot geodesic along tangent vector
    E.append(G.group.exp(v))
    E.append(G.group.exp(-v))
    F.append(G.group.exp(.2 * v))
    F.append(G.group.exp(-.2 * v))
E = jnp.asarray(E)
F = jnp.asarray(F)

env: JAX_PLATFORM_NAME=cpu


We can run the following commands for these data sets.

In [8]:
T_EE = bistat.hotellingT2(E, E)
T_EF = bistat.hotellingT2(E, F)
D_EE = bistat.bhattacharyya(E, E)
D_EF = bistat.bhattacharyya(E, F)

print(f'''
Comparing E to E
Hotelling T² stat.: {T_EE}
Bhatacharyya dist.: {D_EE}

Comparing E to F
Hotelling T² stat.: {T_EF}
Bhatacharyya dist.: {D_EF}
''')


Comparing E to E
Hotelling T² stat.: 0.0
Bhatacharyya dist.: 0.0

Comparing E to F
Hotelling T² stat.: 1.4951966531215333e-13
Bhatacharyya dist.: 2.8665337562561035



As expected the difference from a data set to itself is zero for both indices.
Furthermore, since $E$ and $F$ have mean $I$, their bi-invariant Hotelling $T^2$ statistic is still (numerically) zero.
On the other hand, the Bhatacharyya distance between them is positive, since their covariance structures differ.

We can also test the invariance under translations of both indices. For $\text{SE}(3)$ left and right
translations ('lefttrans' and 'righttrans' in _Morphomatics_) by an element $B \in \text{SE}(3)$ are simply multiplications with B from left and right,
repectively.

In [9]:
# random element
B = G.rand()
# left translate all elements of E and F by B
BE = jax.vmap(G.group.lefttrans, (0, None))(E, B)
BF = jax.vmap(G.group.lefttrans, (0, None))(F, B)
# right translate all elements of E and F by B
EB = jax.vmap(G.group.righttrans, (0, None))(E, B)
FB = jax.vmap(G.group.righttrans, (0, None))(F, B)

Now, we can compute both notions on the left/right translated data sets and compare the results with the original ones.

In [11]:
T_BEBF = bistat.hotellingT2(BE, BF)
T_EBFB = bistat.hotellingT2(EB, FB)
D_BEBF = bistat.bhattacharyya(BE, BF)
D_EBFB = bistat.bhattacharyya(EB, FB)

print(f'''
Difference under left-translation 
Hotelling T² stat.: {jnp.abs(T_BEBF - T_EF)}
Bhatacharyya dist.: {jnp.abs(D_BEBF - D_EF)}

Difference under right-translation 
Hotelling T² stat.: {jnp.abs(T_EBFB - T_EF)}
Bhatacharyya dist.: {jnp.abs(D_EBFB - D_EF)}
''')


Difference under left-translation 
Hotelling T² stat.: 1.5431319416725486e-11
Bhatacharyya dist.: 2.384185791015625e-07

Difference under right-translation 
Hotelling T² stat.: 5.317740171817409e-12
Bhatacharyya dist.: 4.76837158203125e-07



Because of the bi-invariance property both indices give the same values for the translated data.